In [ ]:
import requests
from dotenv import load_dotenv
import os
import numpy as np
import time
import pandas as pd
import base64
import ast

In [ ]:
dfs = []
splitSize = 10

for i in range(1, splitSize+1):
    df = pd.read_csv(f"outputs/github_repos_part{i}.csv")
    dfs.append(df)

fullSet = pd.concat(dfs, ignore_index=True)

i = 0
api_token = os.getenv("API_TOKEN")

In [ ]:
load_dotenv("./token.env")
api_token = os.getenv("API_TOKEN")
while i < len(fullSet):
    time.sleep(1)  # To respect rate limits
    owner = ast.literal_eval(fullSet.iloc[i].owner).get("login")
    repo = fullSet.iloc[i].name

    fullName = fullSet.iloc[i].full_name

    headers = {"Authorization": f"token {api_token}"}

    api_url = f"https://api.github.com/repos/{fullName}/readme"

    try:
        response = requests.get(api_url, headers=headers)
    except: 
        print("Request failed, waiting before retrying")
        time.sleep(60)
        continue
    

    if response.status_code == 200:
        readme_data = response.json()
        if 'content' in readme_data:
            readme_content = base64.b64decode(readme_data['content']).decode('utf-8')
            fullSet.at[i, 'readme'] = readme_content
        else:
            print("No content found in the README response.")
        i = i + 1
        print(f"{i-1} done")
    elif response.status_code == 404:
        print("README not found for this repository.")
        i = i + 1
        print(f"{i-1} done")
    elif response.status_code == 403:
        print("Rate limit exceeded. Waiting before retrying.")
        time.sleep(60)
    else:
        print(f"Called for {api_url}")
        print(f"Failed with status code: {response.status_code}")
        print(f"Response text: {response.text}")
        



In [ ]:
splitSize = 10
split_dfs = np.array_split(fullSet, splitSize)
for i in range(0, splitSize):
    df = split_dfs[i]
    df.to_csv(f"outputs/aug/github_repos_readme_part{i+1}.csv", index=False)